In [2]:
import pandas as pd
import numpy as np

ImportError: cannot import name 'label'

# GUI - Under Construction

This creates the dialogue box where the user enters the project information

Needs to be able to import the customer load file from csv
Inputs:
1) Cost of solar ($/w)
2) Cost of storage ($/kWh)

In [1]:
from tkinter import *

class EnergyModelGUI:
    def __init__(self, master):
        self.master = master
        master.title("Input Project Information")

        self.label = Label(master, text="Solar $/W").grid(row=0,column=0)
        self.entry1 = Entry(master).grid(row=0,column=1)
       
        self.button = Button(master, text="Enter", command=self.greet)
        self.button(column=2)

    def greet(self): #need to set this to capture the entry rather than prompt greeting
        print("Greetings!")

root = Tk()
my_gui = EnergyModelGUI(root)
root.mainloop()

TypeError: 'Button' object is not callable

# Import Datasets: PV Output and Customer Load

Note: Need to find data that better suits this scenario
Note2: Need to automate the PV data pull here
Note3: Make customer load read from excel

In [5]:
#PV Output
PVOutput = pd.read_excel('/Users/ajames2/Desktop/Python/PVOutput.xlsx')

#Customer Load
CustomerLoad = pd.read_excel('/Users/ajames2/Desktop/Python/CustomerLoad.xlsx')

# Scenario Inputs: Diesel, Storage, PV, and Electricity Rates

Note: Need to get all this into Tkinter
Note2: Need to add in dynamic approach for lifetime on diesel and storage based on throughput
Note3: Need to add in roundtrip efficiency
Note4: Add in solar costs and multi-year

In [6]:
#Input Data

#Storage
storage_size_kW = float(250)
storage_hours = float(4)
storage_capex = float(250000)
years_of_use = float(10)
storage_max_throughput = float(2700*storage_size_kW*storage_hours)

#Storage Financing
tenor = 10
interest_rate = 0.20

#Storage Annual Payment
storage_annual_payment = (storage_capex *(interest_rate * (1 + interest_rate) ** tenor) /
                         ((1 + interest_rate) ** tenor - 1))

#solar LCOE
pv_size_kW = 150
pv_cpw = 2.5 #pv cost per watt
pv_capex = (pv_size_kW * pv_cpw * 1000)
pv_irradiation = 1400
pv_years = 20
pv_total_production = pv_irradiation * pv_years * pv_size_kW

#Solar LCOE
solar_lcoe = pv_capex / pv_total_production

#Diesel Generator
diesel_size_kW = float(250)
diesel_capex = float(100000)
diesel_fuel_cost =float(3.5)
diesel_price = float(.40)
diesel_years = 10
diesel_max_throughput = float(36000) #hours
    
#Electricity Price - TOU
        #00:01 - 11:00
        #11:01 - 15:00
        #15:01 - 24:00
    
#Electricity Price - Fixed
electricity_price = float(0.13)

# Clean Datasets and Create New Dataframe

In [7]:
#Rename kWh columns
CustomerLoad.rename(columns={'Building 74 - kWh Total Electricity (kWh)':'Customer_Load'}, inplace=True)

#Convert to hourly
CustomerLoad['timestamp_hour'] = CustomerLoad['Timestamp'].apply(
    lambda x: pd.Timestamp(year=x.year,month=x.month,day=x.day,hour=x.hour))
CustomerLoad.pivot_table(index=['timestamp_hour'], 
                         values='Customer_Load', aggfunc='sum').reset_index()
del CustomerLoad['Timestamp']
EnergyMaster = CustomerLoad

# Create Required Columns for New Dataframe

In [35]:
#Add electricity price
EnergyMaster['electricity_price'] = electricity_price

#create column for pv production
EnergyMaster['PV_Output'] = PVOutput

#create column to show customers load after they have consumed PV
EnergyMaster['PV_Output_Post_Load'] = (EnergyMaster['PV_Output']-EnergyMaster['Customer_Load']).clip_lower(0)

#create column for amount of PV left after customer load
EnergyMaster['Customer_Load_Post_PV'] = (EnergyMaster['Customer_Load']-EnergyMaster['PV_Output']).clip_lower(0)

#create storage state of charge in three parts

    #charge storage first from the remaining PV available
EnergyMaster['starting_storage_SOC_kWh'] = EnergyMaster['PV_Output_Post_Load'].cumsum
    ().clip(0,storage_size_kW * storage_hours)
    
    #identify the potential charge from the grid capped by the storage units size
EnergyMaster['potential_grid_charge'] = ((storage_size_kW * storage_hours) - EnergyMaster['starting_storage_SOC_kWh']).cumsum
    ().clip(0,storage_size_kW)
    
    #charge storage from the grid capped at the storage unit size
EnergyMaster['finished_storage_SOC_kWh'] = (EnergyMaster['starting_storage_SOC_kWh'] + EnergyMaster['potential_grid_charge']).cumsum
    ().clip(0,storage_size_kW * storage_hours)

#create customers load post storage
EnergyMaster['customer_load_post_storage'] = EnergyMaster['Customer_Load_Post_PV'] - EnergyMaster['starting_storage_SOC_kWh']

#EnergyMaster['Storage_SOC_Post_Load'] = (EnergyMaster['Storage_SOC_kWh']-EnergyMaster['Customer_Load_Post_PV']).clip_lower(0)

#Add solar LCOE
EnergyMaster['solar_lcoe'] = solar_lcoe

#Add diesel state of charge
#EnergyMaster['Diesel_SOC'] = EnergyMaster['CustomerLoad'].cumsum().clip(0,diesel_size_kW)

EnergyMaster.head(100)

,Customer_Load,timestamp_hour,electricity_price,PV_Output,PV_Output_Post_Load,Customer_Load_Post_PV,Storage_SOC_kWh,Storage_SOC_Post_Load,solar_lcoe,grid_down_probability,grid_down,cost_in_diesel,diesel_scenario_price,grid_charge,potential_grid_charge,starting_storage_SOC_kWh,finished_storage_SOC_kWh
0,22.8125,2014-01-01 00:00:00,0.13,0.000,0.0000,22.8125,0.000,0.0000,0.089286,0.164863,0,0.4,0.13,250.0,250.0,0.0000,250.0
1,24.6650,2014-01-01 00:00:00,0.13,0.000,0.0000,24.6650,0.000,0.0000,0.089286,0.297530,0,0.4,0.13,250.0,250.0,0.0000,500.0
2,24.1725,2014-01-01 01:00:00,0.13,0.000,0.0000,24.1725,0.000,0.0000,0.089286,0.007840,1,0.4,0.40,250.0,250.0,0.0000,750.0
3,24.9500,2014-01-01 01:00:00,0.13,0.000,0.0000,24.9500,0.000,0.0000,0.089286,0.776753,0,0.4,0.13,250.0,250.0,0.0000,1000.0
4,23.6175,2014-01-01 01:00:00,0.13,0.000,0.0000,23.6175,0.000,0.0000,0.089286,0.920186,0,0.4,0.13,250.0,250.0,0.0000,1000.0
5,23.7425,2014-01-01 01:00:00,0.13,0.000,0.0000,23.7425,0.000,0.0000,0.089286,0.353561,0,0.4,0.13,250.0,250.0,0.0000,1000.0
6,24.7725,2014-01-01 02:00:00,0.13,0.000,0.0000,24.7725,0.000,0.0000,0.089286,0.423067,0,0.4,0.13,250.0,250.0,0.0000,1000.0
7,23.4475,2014-01-01 02:00:00,0.13,19.396,0.0000,4.0515,0.000,0.0000,0.089286,0.721184,0,0.4,0.13,250.0,250.0,0.0000,1000.0
8,24.4975,2014-01-01 02:00:00,0.13,230.275,205.7775,0.0000,411.555,411.5550,0.089286,0.588565,0,0.4,0.13,250.0,250.0,205.7775,1000.0
9,24.4175,2014-01-01 02:00:00,0.13,433.377,408.9595,0.0000,864.737,864.7370,0.089286,0.872919,0,0.4,0.13,250.0,250.0,614.7370,1000.0


# Create Randomized Grid Outages

In [9]:
#variables
days_per_week_grid_down = 3
hoursdown = 4
percent_downtime = days_per_week_grid_down / (7.0*24/hoursdown)

#calculations
EnergyMaster['grid_down_probability'] = EnergyMaster['Customer_Load'].apply(lambda x: np.random.random())
def grid_is_down(rand_num):
    if rand_num < percent_downtime:
        return 1
    else:
        return 0
EnergyMaster['grid_down'] = EnergyMaster['grid_down_probability'].apply(grid_is_down)

# Calculate Combined Price of Diesel Scenario

Note: Storage-only scenario cost is the annual payment lease payment plus electricity price
Note2: Storage and PV cost is the annual lease payment plus the cost of solar plus electricity price

In [12]:
#diesel scenario

EnergyMaster['cost_in_diesel'] = diesel_price
EnergyMaster['diesel_scenario_price'] = EnergyMaster.loc[EnergyMaster['grid_down']<1,['electricity_price','cost_in_diesel']]
EnergyMaster['diesel_scenario_price'].fillna(EnergyMaster['cost_in_diesel'],inplace=True)

EnergyMaster.head(100)

,Customer_Load,timestamp_hour,electricity_price,PV_Output,PV_Output_Post_Load,Customer_Load_Post_PV,Storage_SOC_kWh,Storage_SOC_Post_Load,solar_lcoe,grid_down_probability,grid_down,cost_in_diesel,diesel_scenario_price
0,22.8125,2014-01-01 00:00:00,0.13,0.000,0.0000,22.8125,0.0000,0.0000,0.089286,0.164863,0,0.4,0.13
1,24.6650,2014-01-01 00:00:00,0.13,0.000,0.0000,24.6650,0.0000,0.0000,0.089286,0.297530,0,0.4,0.13
2,24.1725,2014-01-01 01:00:00,0.13,0.000,0.0000,24.1725,0.0000,0.0000,0.089286,0.007840,1,0.4,0.40
3,24.9500,2014-01-01 01:00:00,0.13,0.000,0.0000,24.9500,0.0000,0.0000,0.089286,0.776753,0,0.4,0.13
4,23.6175,2014-01-01 01:00:00,0.13,0.000,0.0000,23.6175,0.0000,0.0000,0.089286,0.920186,0,0.4,0.13
5,23.7425,2014-01-01 01:00:00,0.13,0.000,0.0000,23.7425,0.0000,0.0000,0.089286,0.353561,0,0.4,0.13
6,24.7725,2014-01-01 02:00:00,0.13,0.000,0.0000,24.7725,0.0000,0.0000,0.089286,0.423067,0,0.4,0.13
7,23.4475,2014-01-01 02:00:00,0.13,19.396,0.0000,4.0515,0.0000,0.0000,0.089286,0.721184,0,0.4,0.13
8,24.4975,2014-01-01 02:00:00,0.13,230.275,205.7775,0.0000,205.7775,205.7775,0.089286,0.588565,0,0.4,0.13
9,24.4175,2014-01-01 02:00:00,0.13,433.377,408.9595,0.0000,614.7370,614.7370,0.089286,0.872919,0,0.4,0.13


# Calculate Combined Price of Solar Scenario

In [33]:
EnergyMaster['cost_in_solar'] = solar_lcoe
EnergyMaster['solar_scenario_price'] = EnergyMaster.loc[EnergyMaster['grid_down']<1,['electricity_price','solar_lcoe']]
EnergyMaster['solar_scenario_price'].fillna(EnergyMaster['solar_lcoe'],inplace=True)

year
2014    10773.989286
2015    10259.135714
dtype: float64

# Calculate Totals

In [41]:
#sum by year
EnergyMaster['timestamp_hour'] = pd.to_datetime(EnergyMaster['timestamp_hour'])
EnergyMaster['year'] = EnergyMaster['timestamp_hour'].dt.year

TotalSolarPrice = EnergyMaster.groupby(['year'])['solar_scenario_price'].sum()
TotalDieselPrice = (EnergyMaster.groupby(['year'])['diesel_scenario_price'].sum() + (diesel_capex / diesel_years))
TotalStoragePrice = (EnergyMaster.groupby(['year'])['solar_scenario_price'].sum() + storage_annual_payment)

print()
TotalStoragePrice

year
2014    64074.979935
2015    61045.453506
Name: solar_scenario_price, dtype: float64

# Calculate Throughput 

In [49]:
#storage change in charge

#fix variables with new storage categories above

#EnergyMaster['storage_change_charge'] = EnergyMaster['Storage_SOC'] - EnergyMaster['Storage_SOC_Post_Load']

#storage_throughput = EnergyMaster.groupby(['year'])['storage_change_charge'].sum()
#diesel_throughput = EnergyMaster.groupby(['year'])['grid_down'].sum()

print()
diesel_throughput

year
2014    2491
2015     834
Name: grid_down, dtype: int64

# Optimization

In [ ]:
from scipy import optimize

#variables we control:
hours = range(1,9)
costpergal = [2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
leaserate = [0.15, 0.175, 0.20, 0.225, 0.25]

In [ ]:
# this creates a dataframe of all possible combinations of the variables above:

from itertools import product
costs_df = pd.DataFrame(list(product(hours, daysdown, costpergal,leaserate)), 
                     columns=['hoursdown', 'daysdown','costpergal','leaserate'])
costs_df.head()diesel = []
storage = []

# now, for everyone of those combinations of hours down, days down, costsper gallon etc 
# we will calculate the energy cost for diesel and the energy cost for storage

for index, row in costs_df.iterrows():
    
    #these are the functions we ran above, but now doing them in a more structured fashion
    df = energy.groupby(energy.index / row['hoursdown']).sum() 
    df['grid_down_probability'] = df['kWh'].apply(lambda x: np.random.random())
    percent_downtime = row['daysdown'] / (7.0*24/row['hoursdown']) 
    df['grid_down'] = df['grid_down_probability'].apply(grid_is_down)
    kwh_grid_down = (df['grid_down']*df['kWh']).sum() #this gives us the total kWh that the grid is down

    
    #these calculate the diesel cost in this scenario for every row in the dataframe and appends to a list
    diesel_total_cost = (diesel_annual_payment + #lease costs
                     kwh_grid_down*diesel_om_kwh + #operations and maintenance
                     df['grid_down'].sum()*hoursdown*gallons_per_hour*row['costpergal']) #fuel
    diesel.append(diesel_total_cost)
        
    #now the storage costs   
    storage_total_cost = (storage_annual_payment + kwh_grid_down*storage_electricity_rate)
    storage.append(storage_total_cost)
    
costs_df['dieselcost'] = diesel
costs_df['storagecost'] = storage